
# Source localization with single dipole fit


The aim of this lecture is to show you how to do sequential and
fixed dipole fitting with MNE-Python.

In [1]:
%matplotlib qt
import matplotlib.pyplot as plt

import os
import numpy as np
import mne

mne.set_log_level('WARNING')

# Change the following path to where the folder ds000117-practical is on your disk
data_path = os.path.expanduser("/mnt/c/Users/Egor/Downloads/MEEG/ds000117-practical/")

raw_fname = os.path.join(data_path,
    'derivatives/meg_derivatives/sub-01/ses-meg/meg/sub-01_ses-meg_task-facerecognition_run-01_proc-sss_meg.fif')

epochs_fname = raw_fname.replace('_meg.fif', '-epo.fif')

In [2]:
subjects_dir = os.path.join(data_path, 'derivatives/freesurfer-reconall/')
subject = 'sub-01'
trans_fname = os.path.join(data_path,
    'derivatives/meg_derivatives/sub-01/ses-meg/meg/sub-01-trans.fif')
bem_fname = os.path.join(data_path,
    'derivatives/meg_derivatives/sub-01/ses-meg/meg/sub-01-bem.fif')
fname_surf_lh = os.path.join(subjects_dir, subject, 'surf', 'lh.white')

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
      <li>Check that the head geometry and sensor alignment is correct</li>
    </ul>
</div>

You will need to use the `mne.viz.plot_alignment` function that can take as input a `bem` parameter of type `ConductorModel`

In [3]:
mne.viz.plot_alignment?

In [4]:
bem = mne.bem.read_bem_solution(bem_fname)
bem

<ConductorModel  |  BEM (1 layer)>

In [5]:
# TODO
info = mne.io.read_info(epochs_fname)
fig = mne.viz.plot_alignment(info, trans_fname, subject=subject, dig=True,
                             subjects_dir=subjects_dir, bem=bem, verbose=True);

Could not find the surface for head in the provided BEM model, looking in the subject directory.
Using outer_skin.surf for head surface.
Getting helmet for system 306m


## Let's localize the N170m (using MEG only) using dipole fitting

In [6]:
epochs = mne.read_epochs(epochs_fname, proj=False)
epochs.info['projs'] = epochs.info['projs'][::3]

In [7]:
epochs.pick_types(meg=True, eeg=False)

<EpochsFIF  |   140 events (all good), -0.5 - 2 sec, baseline [-0.2, 0], ~252.8 MB, data loaded,
 'face/famous/first': 24
 'face/famous/immediate': 10
 'face/famous/long': 14
 'face/unfamiliar/first': 24
 'face/unfamiliar/immediate': 9
 'face/unfamiliar/long': 10
 'scrambled/first': 25
 'scrambled/immediate': 13
 'scrambled/long': 11>

In [8]:
cov = mne.compute_covariance(epochs, rank='info', tmax=0, verbose=True)

Computing data rank from raw with rank='info'
    MEG: rank 63 after 5 projectors applied to 306 channels
    Created an SSP operator (subspace dimension = 5)
    Setting small MEG eigenvalues to zero (without PCA)
Reducing data rank from 306 -> 63
Estimating covariance using EMPIRICAL
Done.
Number of samples used : 21140
[done]


EA: Degrees of freedom should be 64...

TODO: look into the empty room method.

In [9]:
cov.plot(epochs.info)

(<Figure size 760x370 with 4 Axes>, <Figure size 760x370 with 2 Axes>)

Gradiometers cancel out a lot of noise, so it should give a lot less noise.

We have a rank deficiency.

When you do SSS, you're combining gradiometers and magnetometers... They will become highly dependent. This can be seen that both types of sensors now have the same rank. We can't do more than 64 dipoles.

In [27]:
epochs.pick_types(meg=True, eeg=False)

<EpochsFIF  |   140 events (all good), -0.5 - 2 sec, baseline [-0.2, 0], ~252.8 MB, data loaded,
 'face/famous/first': 24
 'face/famous/immediate': 10
 'face/famous/long': 14
 'face/unfamiliar/first': 24
 'face/unfamiliar/immediate': 9
 'face/unfamiliar/long': 10
 'scrambled/first': 25
 'scrambled/immediate': 13
 'scrambled/long': 11>

In [10]:
evoked_face = epochs['face'].average()
evoked_scrambled = epochs['scrambled'].average()

In [13]:
contrast = mne.combine_evoked([evoked_face, evoked_scrambled], [0.5, -0.5])
contrast.crop(None, 0.2)
contrast.plot_joint();

In [19]:
contrast.plot_white(cov);

In [32]:
# Fit a dipole using a sequential (time-varying position and orientation) fit
contrast_crop = contrast.copy().crop(0.150, 0.170)
dip, residual = mne.fit_dipole(contrast_crop, cov, bem_fname,
                               trans_fname)
print(dip)

<Dipole  |  n_times : 7, tmin : 0.150, tmax : 0.170>


In [33]:
# Look at our result
print(dip.gof)

[36.34234788 37.48998902 32.86427922 34.41853191 39.88987071 42.26846419
 26.15036641]


This is not a very good fit. Can be improved by using fewer sensors.

In [34]:
dip.plot_locations(subject=subject, trans=trans_fname,
                   subjects_dir=subjects_dir, mode='orthoview');

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
      <li>Try to improve the dipole fit by using a subselection of channels [0] </li>
      <li>What is the improvement in max GOF from using a subselection of channels?</li>     
      <li>Should you also try to maximize GOF?</li>
    </ul>
</div>

Tips and tricks:

   - [0] Subselect channels (think about ``mne.read_selection(...)``, ``evoked.pick_channels(...)``)

In [36]:
# TODO
selection = mne.read_selection('Left', info=contrast.info)
# selection is just a list of channels

# Fit a dipole using a sequential (time-varying position and orientation) fit
dip, residual = \
    mne.fit_dipole(contrast_crop.copy().pick_channels(selection),
                   cov, bem_fname, trans_fname)
print(dip)

<ipython-input-36-c2b3e9e0f626>:8: RuntimeWarning: Projection vector "EOG-planar--0.200-0.200-PCA-01" has magnitude 0.71 (should be unity), applying projector with 102/204 of the original channels available may be dangerous, consider recomputing and adding projection vectors for channels that are eventually used. If this is intentional, consider using info.normalize_proj()
  cov, bem_fname, trans_fname)
<ipython-input-36-c2b3e9e0f626>:8: RuntimeWarning: Projection vector "EOG-planar--0.200-0.200-PCA-04" has magnitude 0.79 (should be unity), applying projector with 102/204 of the original channels available may be dangerous, consider recomputing and adding projection vectors for channels that are eventually used. If this is intentional, consider using info.normalize_proj()
  cov, bem_fname, trans_fname)
<ipython-input-36-c2b3e9e0f626>:8: RuntimeWarning: Projection vector "EOG-axial--0.200-0.200-PCA-03" has magnitude 0.89 (should be unity), applying projector with 51/102 of the original 

<Dipole  |  n_times : 7, tmin : 0.150, tmax : 0.170>


<ipython-input-36-c2b3e9e0f626>:8: RuntimeWarning: Projection vector "EOG-planar--0.200-0.200-PCA-01" has magnitude 0.71 (should be unity), applying projector with 102/204 of the original channels available may be dangerous, consider recomputing and adding projection vectors for channels that are eventually used. If this is intentional, consider using info.normalize_proj()
  cov, bem_fname, trans_fname)
<ipython-input-36-c2b3e9e0f626>:8: RuntimeWarning: Projection vector "EOG-planar--0.200-0.200-PCA-04" has magnitude 0.79 (should be unity), applying projector with 102/204 of the original channels available may be dangerous, consider recomputing and adding projection vectors for channels that are eventually used. If this is intentional, consider using info.normalize_proj()
  cov, bem_fname, trans_fname)
<ipython-input-36-c2b3e9e0f626>:8: RuntimeWarning: Projection vector "EOG-axial--0.200-0.200-PCA-03" has magnitude 0.89 (should be unity), applying projector with 51/102 of the original 

In [37]:
print(dip.gof)

[67.71724856 56.68556272 52.21442333 61.40300611 77.45784748 80.58329749
 71.65020871]


In [38]:
dip.plot_locations(subject=subject, trans=trans_fname,
                   subjects_dir=subjects_dir, mode='orthoview');